In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
from igraph import *
import warnings
warnings.filterwarnings("ignore")
import copy

In [ ]:
from joblib import Parallel,delayed

In [ ]:
#信息传播（父类）
class UATU:
    def __init__(self,alpha,lamda,segma,yta,miu,ypsl,g,g1,time,state1,bian,tao,p,n_time):
        self.alpha=alpha   #沉默者遗忘信息的概率
        self.lamda=lamda   #信息传播率
        self.segma=segma   #变沉默的概率
        self.yta=yta       #可以在子类定义的从一个区域i转移到一个区域j时，调节转移概率与k的参数
        self.beta=0    #流行病传播率                 
        self.miu=miu       #变为恢复者的概率
        self.p=p           #移动概率
        self.ypsl=ypsl     #A/T的个人转移概率改变参数
        self.g=g           #传入的元种群网络结构，为BA网络
        self.g1=g1         #传入的信息层网络结构
        self.time=time     #时间步
        self.bian=bian     #为了多次蒙特卡洛模拟设定的值
        self.tao=tao       #免疫因子
        self.unaw={i:{} for i in range(len(g.vs))}
        self.aware={i:{} for i in range(len(g.vs))}
        self.t_mo={i:{} for i in range(len(g.vs))}
        self.state1={i:{} for i in range(len(g.vs))}    
        self.state_1_change=[]  #存储每一时间步，每个个体的思想状态
        self.count_unaw=[]   #统计每一时间步，无意识者的人数
        self.count_aware=[]  #统计每一时间步，有意识者的人数
        self.count_t=[]       #统计每一时间步，沉默者的人数
        self.count_U=0
        self.count_A=0
        self.count_T=0
        self.zonpeo=len(g1.vs)   #统计总人口
        self.state1=state1       #传入人口初始状态
        self.ntime=n_time       #The information step is a few steps ahead
   
    def init_state(self):
        for i2,j2 in self.state1.items():
             for u,t in j2.items():
                if t[1]=='U':
                     self.unaw[i2][u]=t[1]
                elif t[1]=='A':
                     self.aware[i2][u]=t[1]
                elif t[1]=='R':
                     self.t_mo[i2][u]=t[1] 
                          
    def infomation_diffuse(self):
        self.init_state()
        state_1={i:{} for i in range(len(self.g.vs))}
        for day in range(self.ntime*self.time+self.ntime):
            for i0,j0 in self.unaw.items():
                state_1[i0].update(j0)
                self.count_U+=len(j0)
            for i1,j1 in self.aware.items():    #循环记录初始的无意识、有意识、沉默者
                state_1[i1].update(j1)
                self.count_A+=len(j1)
            for i2,j2 in self.t_mo.items():
                state_1[i2].update(j2)
                self.count_T+=len(j2)
                
            self.count_unaw.append(self.count_U/ self.zonpeo)
            self.count_aware.append(self.count_A/ self.zonpeo)  #存储每天的人数
            self.count_t.append(self.count_T/ self.zonpeo)
            
            #Prepare for the next update
            self.count_U=0
            self.count_A=0
            self.count_T=0
            
            #Put in a dictionary, in line with information updates
            state_2={}
            for i,j in self.unaw.items():
                state_2.update(j)
            for i1,j1 in self.aware.items():
                state_2.update(j1)
            for i2,j2 in self.t_mo.items():
                state_2.update(j2)
                
                
            self.state_1_change.append(copy.deepcopy(state_1))  #Record the updated information status every day
            
            #Store the status of each information update
            temp_A={i:{} for i in range(len(self.g.vs))}
            temp_U={i:{} for i in range(len(self.g.vs))}
            temp_T={i:{} for i in range(len(self.g.vs))}
            #print(state_1)
            
            for i,j in state_1.items():
                for k,l in j.items():
                    #Update from U to A
                    if l=='U':
                        nbs_awareness=sum([1 for x in self.g1.neighbors(k) if state_2[x]=='A'])
                        if nbs_awareness==0:
                            continue
                        if np.random.rand()<(1-(1-self.lamda)**nbs_awareness):
                            temp_A[i][k]=state_1[i][k].replace('U','A')                    
                   #Update from A to T
                    elif l=='A':
                        nbs=0  
                        for b in self.g1.neighbors(k):
                            if state_2[b]=='A' or state_2[b]=='T':
                                nbs+=1
                        if nbs==0:
                            continue
                        if np.random.rand()<(1-(1-self.segma)**nbs):
                            temp_T[i][k]=state_1[i][k].replace('A','T')
                  #Update from T to U
                    elif l=='T':
                        if np.random.rand()<self.alpha:
                            temp_U[i][k]=state_1[i][k].replace('T','U')
    
            for i,j in temp_U.items():
                for y,h1 in j.items():
                    del self.t_mo[i][y]
                    self.unaw[i][y]=temp_U[i][y]
            for i,j in temp_A.items():
                for y,h1 in j.items():
                    del self.unaw[i][y]
                    self.aware[i][y]=temp_A[i][y]
            for i,j in temp_T.items():
                for y,h1 in j.items():
                    del self.aware[i][y]
                    self.t_mo[i][y]=temp_T[i][y]
   
    def save(self):
        self.infomation_diffuse()
        np.save(r'paper_dataset/unaware.npy',self.count_unaw)
        np.save(r'paper_dataset/aware.npy',self.count_aware)
        np.save(r'paper_dataset/aware.npy',self.count_t)
        np.save(r'paper_dataset/aware_everyday.npy',self.state_1_change)
        

In [ ]:
class SIR(UATU):
    def __init__(self,alpha,lamda,segma,yta,miu,ypsl,g,g1,time,state1,bian,tao,p,n_time):
        super().__init__(alpha,lamda,segma,yta,miu,ypsl,g,g1,time,state1,bian,tao,p,n_time)
        self.state1_S={i :{} for i in range(len(self.g.vs))}
        self.state1_I={i :{} for i in range(len(self.g.vs))}
        self.state1_R={i :{} for i in range(len(self.g.vs))}
        self.W=np.zeros([len(self.g.vs),len(self.g.vs)]) #Define the transition probability matrix size
        self.count_S=[]
        self.count_I=[]
        self.count_R=[]
        self.num_S=0
        self.num_I=0
        self.num_R=0
        
    def zygl(self):   #The value of the transfer probability of each element
        for zons in range(len(self.g.vs)):
            for ind in self.g.neighbors(zons):
                self.W[zons][ind]=((self.g.degree(ind)**(self.yta))/sum([self.g.degree(ind)**(self.yta) for ind in self.g.neighbors(zons) ]))
    
    def init_sir(self):
        self.init_state()
        for i,j in self.state1.items():
            for k,v in j.items():
                if v[0]=='S':
                    self.state1_S[i].update({k:v[0]})
                elif v[0]=='I':
                    self.state1_I[i].update({k:v[0]})
                elif v[0]=='R':
                    self.state1_R[i].update({k:v[0]})  
                     
    def epidemic_spread(self,beta,oj):
        self.beta=beta
        self.init_sir()
        self.infomation_diffuse()
        self.zygl()
       
        for day1 in range(self.time):
            state_p={i:{} for i in range(len(self.g.vs))}
            for jw in range(len(self.g.vs)):
                state_p[jw]={**copy.deepcopy(self.state1_S[jw]),**copy.deepcopy(self.state1_I[jw]),**copy.deepcopy(self.state1_R[jw])}
            #disease one step update
            state_zon={i :{} for i in range(len(self.g.vs))}
            for i,j in self.state1.items():
                for k,v in j.items():
                    state_zon[i][k]=state_p[i][k]+self.state_1_change[self.ntime*day1+self.ntime][i][k]   #Every time, the combination of consciousness and disease state
   
            #Record U \ A \ T
            state1_U={i :{} for i in range(len(self.g.vs))}
            state1_A={i :{} for i in range(len(self.g.vs))}
            state1_T={i :{} for i in range(len(self.g.vs))}
            for i,j in state_zon.items():
                for k,v in j.items():
                    if v[1]=='U':
                        state1_U[i].update({k:v})
                                        
                    elif v[1]=='A':
                        state1_A[i].update({k:v})
                        
                    elif v[1]=='T':
                        state1_T[i].update({k:v})
            #Record the density of S \ i \ r every time            
            self.num_S=0
            self.num_I=0
            self.num_R=0
            for i,j in state_zon.items():
                for k,v in j.items():
                    if v[0]=='S':
                        self.num_S+=1
                    elif v[0]=='I':
                        self.num_I+=1
                    elif v[0]=='R':
                        self.num_R+=1
            self.count_S.append(self.num_S/self.zonpeo)
            self.count_I.append(self.num_I/self.zonpeo)
            self.count_R.append(self.num_R/self.zonpeo)
           # the number of individuals that moved from node j o node i:
            zhuan_dao={i1:{} for i1 in range(len(self.g.vs))}
            zhuan_zou={i1:{} for i1 in range(len(self.g.vs))}  
            c_stor={i:{} for i in range(len(self.g.vs))}
            temp={i1:{} for i1 in range(len(self.g.vs))}
            
            for iu in state1_U.keys():
                for cq in range(len(self.g.vs)):
                    count=0
                    if self.W[:,iu][cq]==0:
                        continue
                    else:
                        for ku, lu in state1_U[cq].items():
                            if np.random.rand()<=self.p*self.W[:,iu][cq]:
                                count+=1
                                zhuan_dao[iu].update({ku:lu})
                                temp[cq].update({ku:lu})
                                zhuan_zou[cq].update({ku:str(iu)+str(lu)})
                            else:
                                continue
                        for ko,lo in state1_A[cq].items():
                            if np.random.rand()<=(self.p**(self.ypsl))*self.W[:,iu][cq]:
                                zhuan_dao[iu].update({ko:lo})
                                count+=1
                                temp[cq].update({ko:lo})
                                zhuan_zou[cq].update({ko:str(iu)+str(lo)})
                            else:
                                continue
                        for kp,lp in state1_T[cq].items():
                            if np.random.rand()<=(self.p**(self.ypsl))*self.W[:,iu][cq]:
                                zhuan_dao[iu].update({kp:lp})
                                count+=1
                                temp[cq].update({kp:lp})
                                zhuan_zou[cq].update({kp:str(iu)+str(lp)})
                            else:
                                continue
                        c_stor[iu].update({cq:count})
                        
            for qd,qa in temp.items():                  #这里是原始的SIR删除掉转移走的
                for qc,qo in qa.items():
                    if qo[0]=='S':
                        del self.state1_S[qd][qc]
                    elif qo[0]=='I':
                        del self.state1_I[qd][qc]
                    elif qo[0]=='R':
                        del self.state1_R[qd][qc]
                        
            state_o={i:{} for i in range(len(self.g.vs))}
            for qw in range(len(self.g.vs)):
                state_o[qw]={**self.state1_S[qw],**self.state1_I[qw],**self.state1_R[qw]} 
            infect={}
            infect_mi={}
            # prob_ben=[]
            state_I_temp={i:{} for i in range(len(self.g.vs))}
            state_R_temp={i:{} for i in range(len(self.g.vs))}
            state_he={i:{} for i in range(len(self.g.vs))}
            
            for j1,k1 in state_zon.items():
                count_a=0
                for qw,qe in k1.items():
                    if qe[0]=='I':
                        count_a+=1
                infect[j1]=count_a
                
            for j2 in range(len(self.g.vs)):
                infect_mi[j2]=infect[j2]/len(self.state1[j2])    ### 这里要仔细考虑了
            #The probability of s change i
            for i,j in self.state1_S.items():
                prob_ben=[]
                prob_bena=[]
                prob_ben.append((1-self.beta*(infect_mi[i]))**len(state_o[i]))  #The probability of Su being infected in  own area
                prob_bena.append((1-self.beta*self.tao*(infect_mi[i]))**len(state_o[i]))  #The probability of SA being infected in  own area
                for hy,hu in c_stor[i].items():
                    prob_ben.append((1-self.beta*(infect_mi[hy]))**hu)     #The probability of being infected in one's own region by an infected person from another region
                    prob_bena.append((1-self.beta*self.tao*(infect_mi[hy]))**hu)
                for ku,ki in j.items():
                    if state_zon[i][ku][1]=='A' or state_zon[i][ku][1]=='T':
                        if np.random.rand()<1-np.prod(prob_bena):   #np.prod()Total probability of infection
                            state_I_temp[i][ku]='I'
                    elif state_zon[i][ku][1]=='U':
                        if np.random.rand()<1-np.prod(prob_ben):
                            state_I_temp[i][ku]='I'
                            
            for i2, j2 in self.state1_I.items():
                for k2,l2 in j2.items():
                    if np.random.rand()<self.miu:
                        state_R_temp[i2][k2]='R'
            #the probability that this individual catches the disease when moving to any neighbour of i. 
            for i1,j1 in zhuan_zou.items():
                #这里要注意一下
                for y1,u1 in j1.items():
                    prob_wai=[]
                    prob_waia=[]
                    if u1[-2]=='R':
                        continue
                    elif u1[-2]=='S':
                        if len(u1)==3:
                            if state_zon[i1][y1][1]=='A' or state_zon[i1][y1][1]=='T':
                                prob_waia.append((1-self.beta*self.tao*infect_mi[int(u1[0])])**len(state_o[int(u1[0])]))
                            
                                for wai,waj in c_stor[int(u1[0])].items():
                                    prob_waia.append((1-self.beta*self.tao*infect_mi[wai])**waj)
                                if np.random.rand()<1-np.prod(prob_waia):
                                    state_I_temp[i1][y1]='I'
                            elif state_zon[i1][y1][1]=='U':
                                prob_wai.append((1-self.beta*infect_mi[int(u1[0])])**len(state_o[int(u1[0])]))
                                for wai,waj in c_stor[int(u1[0])].items():
                                    prob_wai.append((1-self.beta*infect_mi[wai])**waj)
                                if np.random.rand()<1-np.prod(prob_wai):
                                    state_I_temp[i1][y1]='I'
                        elif len(u1)==4:
                            if state_zon[i1][y1][1]=='A' or state_zon[i1][y1][1]=='T':
                                prob_waia.append((1-self.beta*self.tao*infect_mi[int(u1[:2])])**len(state_o[int(u1[:2])]))
                                for wai,waj in c_stor[int(u1[:2])].items():
                                    prob_waia.append((1-self.beta*self.tao*infect_mi[wai])**waj)
                                if np.random.rand()<1-np.prod(prob_waia):
                                    state_I_temp[i1][y1]='I'
                            elif state_zon[i1][y1][1]=='U':
                                prob_wai.append((1-self.beta*infect_mi[int(u1[:2])])**len(state_o[int(u1[:2])]))
                                for wai,waj in c_stor[int(u1[:2])].items():
                                    prob_wai.append((1-self.beta*infect_mi[wai])**waj)
                                if np.random.rand()<1-np.prod(prob_wai):
                                    state_I_temp[i1][y1]='I'
                                            
                    elif u1[-2]=='I':
                        if np.random.rand()<self.miu:
                            state_R_temp[i1][y1]='R'
                                            
            for i in range(len(self.g.vs)):
                state_he[i]={**copy.deepcopy(self.state1_S[i]),**copy.deepcopy(self.state1_I[i]),**copy.deepcopy(self.state1_R[i]),**copy.deepcopy(temp[i])}                                 
            self.state1_S={i :{} for i in range(len(self.g.vs))}
            self.state1_I={i :{} for i in range(len(self.g.vs))}
            self.state1_R={i :{} for i in range(len(self.g.vs))}
            for i,j in state_he.items():
                for k,v in j.items():
                    if v[0]=='S':
                        self.state1_S[i].update({k:v[0]})
                    elif v[0]=='I':
                        self.state1_I[i].update({k:v[0]})
                    elif v[0]=='R':
                        self.state1_R[i].update({k:v[0]}) 
                                            
            for iq,jq in state_I_temp.items():
                for fq,dq in jq.items():
                    del self.state1_S[iq][fq]
                    self.state1_I[iq].update({fq:dq})
                                            
            for ir,jr in state_R_temp.items():
                for fr,dr in jr.items():
                    del self.state1_I[ir][ fr]
                    self.state1_R[ir].update({fr:dr})
        return self.count_S,self.count_I,self.count_R     
    # def save_data(self):
    #     # self.bian=na
    #     # self.beta=beta
    #     self.epidemic_spread()
    #     np.save(fr'paper_dataset/change_beta/susceptible{self.bian}.npy',self.count_S)
    #     np.save(fr'paper_dataset/change_beta/infect{self.bian}.npy',self.count_I)
    #     np.save(fr'paper_dataset/change_beta/recover{self.bian}.npy',self.count_R)
      

In [ ]:
state1=np.load('state1.npy',allow_pickle=True).tolist()

In [ ]:
g=Graph()
# g=g.Barabasi(100,2)
g1=Graph()
# g1=g1.Barabasi(sum(c),20)
g=g.Read_Edgelist('edge_list')
g1=g1.Read_Edgelist('edge_list_info')#Execute this cell to import the scale-free network structure


In [ ]:
g1=g1.Read_Edgelist('edge_ER_4')#Execute this cell to import the ER network structure

In [ ]:
c_o=np.linspace(0,0.1,11)#Information transmission rate
c_o=np.append(c_o,0.2)
c_o=np.append(c_o,0.3)
data_stor=[]#
for bianlian in c_o:
    moni=SIR(alpha=(1/14),lamda=(bianlian),segma=(0.3/30),yta=0.5,miu=1/14,p=0.3,ypsl=2,g=g,g1=g1,time=300,state1=state1,bian=1,tao=0.5,n_time=1)
    data=Parallel(n_jobs=40)(delayed(moni.epidemic_spread)(beta,oj) for beta in np.linspace(0.0001,0.003,30) for oj in range(50))
    data_stor.append(data)

In [ ]:
np.save('sf.npy',data_stor)  #stor value